# notifying changes in computational notebooks

blind and low vision jupyter users frequently ask for audible (non-visual) announcements about changes to the state of the document. this demand introduces a new visual and nonvisual feature to computational notebooks that notify any user of an update.

this feature request starts as a nonvisual componment, but quickly we realize that a visual component would be assistive to all users.

[toast components](https://www.scottohara.me/blog/2019/07/08/a-toast-to-a11y-toasts.html) are common patterns for these kinds of notifications, but there are accessibility challenges. one specific challenge is that the aria live message could be destroyed it is announced. this demands the need for a more persistent message, but not immortal. the time delay should be configurable on the order of minutes. 

another notification pattern to consider would use the [notifications api](https://developer.mozilla.org/en-US/docs/Web/API/Notifications_API/Using_the_Notifications_API). this approach would be for escalated notifications, or tens of minute notifications. the os will handle the accessibility of these messages.

[jump to the demo](#log)

In [1]:
from nbconvert_a11y.tables import get_table, Config, new
shell.tangle.parser = midgy.language.python.Python()

In [2]:
%%
## creating a scenario that would be announced in a log

our approach maps [logging levels](https://stackoverflow.com/questions/2031163/when-to-use-the-different-log-levels) - the visual log -
to [aria live](https://www.sarasoueidan.com/blog/accessible-notifications-with-aria-live-regions-part-1/) values - the nonvisual log[^log].
ultimately, we create a reference visual and non-visual logging component that will provide updates to assistive technology.

    from logging import FATAL, CRITICAL, ERROR, WARN, INFO, DEBUG
    FATAL / CRITICAL, ERROR, WARN, INFO, DEBUG;
    activities:\
is a list of actions that may be encountered during interactive computing.
the list provides a log level, message, and call to action.\

    =\
1. info
    : starting python kernel
    : - show kernel info
2. info
    : python kernel started
    : - execute cells
3. info
    : cell 2 executed
    : - cancel cell 2 execution
4. info
    : cell 2 finished successfully
    : - jump to cell output
5. info
    : cell 7 executed
    : - cancel cell 7 execution
6. error
    : cell 7 failed with TypeError
    : - jump to cell 7 traceback
7. critical
    : python kernel restarting
    : - queue cell execution
8. critical
    : python kernel restarted
    : - execute cells

[^log]: `log` is an aria role meant to mimic this pattern. support is generally better for `role=status` or `role=alert`.

In [3]:
%%
    logging_roles:\
is a mapping of logging levels to aria live roles.\

    =\
```yaml
ERROR: assertive
CRITICAL: assertive
INFO: polite
WARN: none
DEBUG: none
```

these levels should be configured in the user interface especially to meet the needs of developers using assistive technologies.

## creating a dataframe about our synthetic scenario

In [4]:
%%
<details>
<summary>dataframe tidying</summary>

```ipython
df = pipe(
    activities, partial(re.sub, "\s+:", ":"), 
    partial(re.sub, "[0-9].\s+|\s-\s", ""), str.splitlines, Series
).str.partition(": ")[[0, 2]].rename(columns=pipe(
    "level message".split(), partial(zip, range(0, 3, 2)), dict, 
))

df["timestamp"] =df.level.apply(
    lambda x: random.randint(1, 10)
).cumsum().pipe(
    pandas.to_datetime, unit="s"
)
df = df.set_index("timestamp")
df["level"] = df["level"].str.upper()

df["live"] = df["level"].apply(logging_roles.get)
df = df.drop(columns="message").assign(
    **df.message.str.partition(":")[[0, 2]].rename(columns=pipe(
        "message action".split(), partial(zip, range(0, 3, 2)), dict, 
    ))
)
df["id"] = [uuid.uuid4() for _ in range(len(df))]

pipe(df, partial(get_table, config=Config(), caption="information needed to construct an accessible table representation"), display)
```
</details>


timestamp,level,live,message,action,id
1970-01-01 00:00:06,INFO,polite,starting python kernel,show kernel info,e1a85cab-418f-42d3-877f-dfdaba82f10d
1970-01-01 00:00:14,INFO,polite,python kernel started,execute cells,fd2d69e7-92d0-40a6-81b4-ea946c5ac708
1970-01-01 00:00:24,INFO,polite,cell 2 executed,cancel cell 2 execution,d72efb84-7fd8-4f23-8beb-0c245df92998
1970-01-01 00:00:29,INFO,polite,cell 2 finished successfully,jump to cell output,3976b44a-9fa6-408a-8672-f74c57372860
1970-01-01 00:00:33,INFO,polite,cell 7 executed,cancel cell 7 execution,6500b07a-749f-4aa6-af04-f797403f4fcb
1970-01-01 00:00:41,ERROR,assertive,cell 7 failed with TypeError,jump to cell 7 traceback,bef39ce7-4091-46de-ba96-4a4b533ae9f9
1970-01-01 00:00:43,CRITICAL,assertive,python kernel restarting,queue cell execution,5b5757dd-7e1f-4076-9b35-9dbbff26a897
1970-01-01 00:00:44,CRITICAL,assertive,python kernel restarted,execute cells,3b290298-41f2-4bef-a3d3-de4070bd86fc
1970-01-01 00:00:47,,None,,,2b32a677-085d-4806-aec0-07e7e0a298f3
1970-01-01 00:00:53,[^LOG],None,`log` is an aria role meant to mimic this pattern. support is generally better for `role=status` or `role=alert`.,,4c6d9229-5c28-43e1-882b-91c660733a20


## accessible table components

create components that convert the wide data above to a user interface component that can be tested.

In [5]:
# https://developer.mozilla.org/en-US/docs/Web/HTML/Element/time

def get_timestamp(s):
    t = new("time", attrs=dict(datetime=s.name.isoformat()))
    # t.append(s.name.strftime('%Y-%m-%d %X'))
    t.append(s.name.strftime('%X'))
    return t

there are a lot of [shenangins to consider when implementing aria live across browsers](https://a11ysupport.io/tech/aria/aria-live_attribute).
we take an html5 forward design approach and restrict ourselves to the nuance around the native output element;
we take heavy advice from [scott o'hara's html `output` analysis](https://scottaohara.github.io/tests/html-output/). 

In [6]:
def get_message(s):
    object = new("output", attrs=dict(
        role="status", id=s.loc["id"], 
        onload="alert(11);",  **{"aria-live": s.live, "data-message": s.message}
    ))
    # object.append(s.message)
    return object

aria live seems most reliable when the text is injected directly into the element. to ensure `aria-live` announces we must create an output element THEN dynamically text content to trigger an announcement.

In [7]:
def get_script(s):
    script = new("script")
    script.append(F""" setTimeout(() => {{
            var out = document.getElementById("{s.id}");
            out.parentElement.parentElement.classList.remove("nv");
            out.textContent = out.dataset.message}}, {s.name.second * 1000})""")
    return script

`get_priority` is the visual log level.

In [8]:
def get_priority(s):
    label = new("label", attrs={"for": s.loc["id"]})
    label.append(s.loc["level"])
    return label

a visual convention with toast notifications is to provide a call to action with a notification.
we couple these visual concepts with non visual concepts.
jupyter has toast notifications, but they would be compliments the activity log.
the activity log is the domain of visual and nonvisual actions.

In [9]:
def get_action(s):
    # might return a link or a button
    butt = new("a" if s.action.startswith(("jump",)) else "button")
    butt.append(s.action)
    return butt

an example activity log `table` that reveals it self and has aria live capabilities.
this is a first demonstration of a more robust activity log for visual and nonvisual users.
the table operates as a keyboard navigable component on assistive technology.

In [10]:
table = (t := df.apply(
    lambda s: Series(dict(
        priority=get_priority(s),
        message=get_message(s),
        action=get_action(s), 
        timestamp=get_timestamp(s), script=get_script(s)
    )), axis=1
)).reset_index(drop=True).set_index("timestamp").pipe(get_table, id="log", **{"aria-labelledby": "log-nm"})
[x.attrs.update(**{"class": "nv"}) for x in table.select("tr")];

the table doesn't need to be seen, but it can not be removed from the visual domain otherwise announcements will be suppressed.
we can use the `details` tag and the non visual css styling to address this.
by wrapping all of the following in a region we expose a landmark for the activity log enhancing discovery.

In [11]:
section = new("section", attrs={"aria-labelledby": "log-nm"})
section.append(details := new("details", attrs=dict(open="")))
details.append(summary := new("summary", attrs=dict(id="log-nm")))
summary.append("activity log")
section.append(table)
section

timestamp,priority,message,action,script
00:00:06,INFO,,show kernel info,
00:00:14,INFO,,execute cells,
00:00:24,INFO,,cancel cell 2 execution,
00:00:29,INFO,,jump to cell output,
00:00:33,INFO,,cancel cell 7 execution,
00:00:41,ERROR,,jump to cell 7 traceback,
00:00:43,CRITICAL,,queue cell execution,
00:00:44,CRITICAL,,execute cells,
00:00:47,,,,
00:00:53,[^LOG],,,


## conclusion

it might be possible to create toast like behavior with persistance using a table. the row of a table could have css applied to appear as a toast message before returning back to its table.

the number of messages will grow, and it will be necessary to cull and throttle the messages. it will be important to do research into disabling/enabling aria live by changing attributes; in other words, it might be possible, in some stacks, to suppress a queued live message.

## styling

In [13]:
%%
<figure>
<figcaption>
<a href="https://www.tpgi.com/the-anatomy-of-visually-hidden/">visually hidden css</a></figcaption>
<blockquote cite="https://www.tpgi.com/the-anatomy-of-visually-hidden/">
    
    display\
```css
details:not([open]) + table, .nv, .non-visual {
    clip: rect(0 0 0 0);
    clip-path: inset(50%);
    height: 1px;
    overflow: hidden;
    position: absolute;
    white-space: nowrap;
    width: 1px;
}
/** demo button **/
a[href="#log"] {
    display: block;
    height: 300px;
    width: 400px;
    background: -moz-element(#log);
    background-size: content;
    background-repeat: no-repeat;
    font-size: 4rem;
}
```
</blockquote>
</figure>